# Interpretable_MLmodel_urban_flood_susceptibility

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from scipy.sparse.linalg import cg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import os
import sys
import glob
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.tree 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import math
import torch
# import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import lime.lime_tabular
import shap
# from interpret_community.tabular_explainer import TabularExplainer
from captum.attr import IntegratedGradients
# import tensorflow as tf
# from tensorflow.keras import backend as K
import time
import lime.lime_tabular
import shap
# from interpret_community.tabular_explainer import TabularExplainer
from captum.attr import IntegratedGradients
# import tensorflow as tf
# from tensorflow.keras import backend as K
import time
from sklearn.preprocessing import MinMaxScaler



In [ ]:
import random
m=32
random.seed(m)
np.random.seed(m)
torch.manual_seed(m)

In [ ]:
# 读取数据
train_data_original=pd.read_csv('train_data_231017_两类事件.csv')
test_data_original=pd.read_csv('test_data_231017_两类事件.csv')

In [ ]:
train_data_original

In [ ]:
test_data_original

In [ ]:
# 输入特征归一化
# 假设 train_data 是你的数据框
scaler = MinMaxScaler()
train_data = train_data_original.copy()  # 创建一个train_data的副本以保留原始数据
train_data.iloc[:, 1:9] = scaler.fit_transform(train_data_original.iloc[:, 1:9])  # 归一化第一列到第12列的数据

test_data = test_data_original.copy()  # 创建一个train_data的副本以保留原始数据
test_data.iloc[:, 1:9] = scaler.fit_transform(test_data_original.iloc[:, 1:9])  # 归一化第一列到第12列的数据

train_data


In [ ]:
test_data

In [ ]:
# 查看 class 列不同值及其个数
class_counts = train_data['Suscept_3'].value_counts()
print('label 列不同值及其个数为：\n', class_counts)
train_data

In [ ]:
# 查看 class 列不同值及其个数
class_counts = test_data['Suscept_3'].value_counts()
print('suscept_3 列不同值及其个数为：\n', class_counts)
test_data

In [ ]:
# 将训练集和测试集合并起来
data_train_test=pd.concat([train_data,test_data])
df=data_train_test.iloc[:,1:-1]
df

In [ ]:
# 所有特征

all_features = df.iloc[:, :-1]

all_features

In [ ]:
train_data

In [ ]:
# 训练集特征输入
X_train = train_data.iloc[:,1:-1]

X_train

In [ ]:
Y_train = train_data['Suscept_3']

Y_train

In [ ]:
X_test = test_data.iloc[:,1:-2]

X_test

In [ ]:
Y_test = test_data['Suscept_3']
Y_test

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

print(pd.value_counts(Y_train))

print(pd.value_counts(Y_test))

In [ ]:
def print_test_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

def print_train_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_train) == Y_train)/len(Y_train)))
    time.sleep(0.5) # to let the print get out before any progress bars
shap.initjs()

## 输入数据统计信息

In [ ]:
all_features.describe()

## 2. Model_training

### 2.1 Xgboost

In [ ]:
'''
# 定义XGBoost模型
xgb_model = xgb.XGBClassifier()
    

# 定义参数空间
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1],
}

# 定义5折交叉验证策略
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 使用GridSearchCV进行自动调参和交叉验证
grid_search_xgb = GridSearchCV(xgb_model,
                           param_grid=param_grid,
                           scoring='recall',
                           cv=cv)

grid_search_xgb.fit(X_train, Y_train)

# 输出最优参数和最优得分
print("Best parameters: ", grid_search_xgb.best_params_)
print("Best score: ", grid_search_xgb.best_score_)

# 用网格搜索到的最优参数重新训练模型
params_xgb =grid_search_xgb.best_params_
best_params_xgb = xgb.XGBClassifier(**params_xgb)
best_params_xgb.fit(X_train, Y_train)
'''

In [ ]:

# xgb模型训练
# 因为优化后之间训练模型就会在解释的时候，SHAP方法调用失败。因此，将优化好的参数重新训练模型
# Best parameters:  {'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9}
# Best score:  0.5849992669597436

# Best parameters:  {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
# Best score:  0.8072517369212393
# model_xgb = xgboost.XGBRegressor('colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8).fit(X_train, Y_train)
# {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
#Best score:  0.808133275323328
# 'colsample_bytree': 1, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9
# 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9
# 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8
# Best parameters:  {'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9}

# 231017:'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9
# 231025:'colsample_bytree': 1, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9
model_xgb_new = xgb.XGBClassifier(colsample_bytree=1, gamma= 0.2, learning_rate= 0.1, max_depth= 7, n_estimators= 200, subsample= 0.9)

# train model on training set
model_xgb_new.fit(X_train, Y_train)


### 2.2 MLP

In [ ]:
'''
# nn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# 定义多层感知机模型
mlp = MLPClassifier()

# 定义参数网格
param_grid = {
    
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    #'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['relu', 'tanh'],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [100, 200, 500],
    #'alpha': [0.0001, 0.05],
    
}

# 定义5折交叉验证器
cv = StratifiedKFold(n_splits=5, shuffle=True,random_state=42
)

# 定义网格搜索器
grid_search_mlp = GridSearchCV(mlp, param_grid=param_grid,
                           cv=cv, scoring='recall')

# 训练模型并进行自动调参
grid_search_mlp.fit(X_train, Y_train)

# 输出最佳参数组合和对应的AUC值
print('Best parameters:', grid_search_mlp.best_params_)
print('Best AUC:', grid_search_mlp.best_score_)

# 输出最优参数和最优得分
print("Best parameters: ", grid_search_mlp.best_params_)
print("Best score: ", grid_search_mlp.best_score_)

# 用网格搜索到的nn最优参数重新训练模型

params_mlp = grid_search_mlp.best_params_
best_params_mlp = MLPClassifier(**params_mlp)
best_params_mlp.fit(X_train, Y_train)
'''

In [ ]:

# nn
# Best parameters: {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'max_iter': 500, 'solver': 'adam'}
#Best parameters:  {'activation': 'logistic', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling', 'max_iter': 200, 'solver': 'adam'}
#Best score:  0.7586361913045934
# 'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 500, 'solver': 'adam'
# 'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'adam'
# 'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'max_iter': 500, 'solver': 'adam'

# 231017:'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'adam'
# 231025:'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 500, 'solver': 'adam'
model_mlp_new = MLPClassifier( activation= 'tanh', hidden_layer_sizes= (100,), learning_rate= 'constant', max_iter= 500, solver= 'adam')

# train model on training set
model_mlp_new.fit(X_train, Y_train)



### 2.3 LR

In [ ]:
'''
# LR
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd

# 定义逻辑回归模型和参数网格
lr = LogisticRegression()
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 500],
}

# 使用GridSearchCV进行自动调参和5折交叉验证
grid_search_lr = GridSearchCV(lr, param_grid, scoring='recall', cv=cv)
grid_search_lr.fit(X_train, Y_train)

# 输出最优参数和最优得分
print("Best parameters: ", grid_search_lr.best_params_)
print("Best score: ", grid_search_lr.best_score_)

# 重新训练模型
params_lr = grid_search_lr.best_params_
best_params_lr =  LogisticRegression(**params_lr)
best_params_lr.fit(X_train, Y_train)

'''

In [ ]:

# lr
# 'C': 10, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'
#Best parameters:  {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
# 'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'
# 'C': 10, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'
# 'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'

# 231017:'C': 10, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'
# 231025:'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'
model_lr_new = LogisticRegression(C= 10, max_iter= 100, penalty= 'l1', solver= 'liblinear')

# train model on training set
model_lr_new.fit(X_train, Y_train)


In [ ]:
'''
# 定义随机森林模型
rf = RandomForestClassifier()

# 定义参数网格
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# 定义交叉验证方法
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 定义网格搜索对象
grid_search_rf = GridSearchCV(rf, param_grid, scoring='recall', cv=cv)

# 训练模型并进行网格搜索
grid_search_rf.fit(X_train, Y_train)

# 输出最优参数和最优得分
print("Best parameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)

# 重新训练模型
params_rf = grid_search_rf.best_params_
best_params_rf =  RandomForestClassifier(**params_rf)
best_params_rf.fit(X_train, Y_train)
'''

In [ ]:

# rf
#Best parameters:  {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
# 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200
# 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50
# 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200

# 231017:'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100
# 231025:'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200
model_rf_new = RandomForestClassifier(max_depth= 20, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 200)

# train model on training set
model_rf_new.fit(X_train, Y_train)


## 3. Model_preformance

In [ ]:
# 将LR, XGB, NN的模型性能进行对比

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc

# 训练三个机器学习模型并预测测试集

model_lr = model_lr_new
model_xgb =  model_xgb_new
model_mlp =  model_mlp_new
model_rf =  model_rf_new

'''

model_lr = best_params_lr
model_xgb =  best_params_xgb
model_mlp =  best_params_mlp
model_rf =  best_params_rf
'''


Y_test_pred1 = model_lr.predict(X_test)
Y_test_pred2 = model_xgb.predict(X_test)
Y_test_pred3 = model_mlp.predict(X_test)
Y_test_pred4 = model_rf.predict(X_test)

# 计算评价指标
accuracy_train1 = accuracy_score(Y_train, model_lr.predict(X_train))
accuracy_train2 = accuracy_score(Y_train, model_xgb.predict(X_train))
accuracy_train3 = accuracy_score(Y_train, model_mlp.predict(X_train))
accuracy_train4 = accuracy_score(Y_train, model_rf.predict(X_train))

accuracy_test1 = accuracy_score(Y_test, model_lr.predict(X_test))
accuracy_test2 = accuracy_score(Y_test, model_xgb.predict(X_test))
accuracy_test3 = accuracy_score(Y_test, model_mlp.predict(X_test))
accuracy_test4 = accuracy_score(Y_test, model_rf.predict(X_test))

precision_train1 = precision_score(Y_train, model_lr.predict(X_train))
precision_train2 = precision_score(Y_train, model_xgb.predict(X_train))
precision_train3 = precision_score(Y_train, model_mlp.predict(X_train))
precision_train4 = precision_score(Y_train, model_rf.predict(X_train))

precision_test1 = precision_score(Y_test, Y_test_pred1)
precision_test2 = precision_score(Y_test, Y_test_pred2)
precision_test3 = precision_score(Y_test, Y_test_pred3)
precision_test4 = precision_score(Y_test, Y_test_pred4)

recall_train1 = recall_score(Y_train, model_lr.predict(X_train))
recall_train2 = recall_score(Y_train, model_xgb.predict(X_train))
recall_train3 = recall_score(Y_train, model_mlp.predict(X_train))
recall_train4 = recall_score(Y_train, model_rf.predict(X_train))

recall_test1 = recall_score(Y_test, Y_test_pred1)
recall_test2 = recall_score(Y_test, Y_test_pred2)
recall_test3 = recall_score(Y_test, Y_test_pred3)
recall_test4 = recall_score(Y_test, Y_test_pred4)

f1score_train1 = f1_score(Y_train, model_lr.predict(X_train))
f1score_train2 = f1_score(Y_train, model_xgb.predict(X_train))
f1score_train3 = f1_score(Y_train, model_mlp.predict(X_train))
f1score_train4 = f1_score(Y_train, model_rf.predict(X_train))

f1score_test1 = f1_score(Y_test, Y_test_pred1)
f1score_test2 = f1_score(Y_test, Y_test_pred2)
f1score_test3 = f1_score(Y_test, Y_test_pred3)
f1score_test4 = f1_score(Y_test, Y_test_pred4)

# 将评价指标存储在字典中
results_train = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (train)': [accuracy_train1, accuracy_train2, accuracy_train3],
    'Precision (train)': [precision_train1, precision_train2, precision_train3],
    'Recall (train)': [recall_train1, recall_train2, recall_train3],
    'F1 score (train)': [f1score_train1, f1score_train2, f1score_train3],
}
results_test = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (test)': [accuracy_test1, accuracy_test2, accuracy_test3],
    'Precision (test)': [precision_test1, precision_test2, precision_test3],
    'Recall (test)': [recall_test1, recall_test2, recall_test3],
    'F1 score (test)': [f1score_test1, f1score_test2, f1score_test3],
}
# 将字典转换为 Pandas 数据框
df_results_train = pd.DataFrame(results_train)
df_results_test = pd.DataFrame(results_test)


In [ ]:
# 将评价指标存储在字典中
results_train_rf = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp','Model_rf'],
    'Accuracy (train)': [accuracy_train1, accuracy_train2, accuracy_train3, accuracy_train4],
    'Precision (train)': [precision_train1, precision_train2, precision_train3,precision_train4],
    'Recall (train)': [recall_train1, recall_train2, recall_train3,recall_train4],
    'F1 score (train)': [f1score_train1, f1score_train2, f1score_train3,f1score_train4],
}
results_test_rf = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp','Model_rf'],
    'Accuracy (test)': [accuracy_test1, accuracy_test2, accuracy_test3,accuracy_test4],
    'Precision (test)': [precision_test1, precision_test2, precision_test3, precision_test4],
    'Recall (test)': [recall_test1, recall_test2, recall_test3,recall_test4],
    'F1 score (test)': [f1score_test1, f1score_test2, f1score_test3,f1score_test4],
}
# 将字典转换为 Pandas 数据框
df_results_train_rf = pd.DataFrame(results_train_rf)
df_results_test_rf = pd.DataFrame(results_test_rf)

In [ ]:
df_results_train

In [ ]:
df_results_test

---

In [ ]:
df_results_train_rf

In [ ]:
df_results_test_rf

In [ ]:
# Compute performance metrics
Y_test1 = Y_test.ravel() # 将Y_test1转换为一维数组
xgb_loss = sklearn.metrics.log_loss(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_loss = sklearn.metrics.log_loss(Y_test1, model_lr.predict_proba(X_test)[:,1])
nn_loss = sklearn.metrics.log_loss(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_loss = sklearn.metrics.log_loss(Y_test1, model_rf.predict_proba(X_test)[:,1])

# constant_loss = sklearn.metrics.log_loss(Y_test1, np.zeros(len(Y_test1)) * y_strain.mean())
print("XGBoost_loss", xgb_loss)
print("LR_loss", linear_loss)
print("MLP_loss", nn_loss)
print("RF_loss", nn_loss)
print()

xgb_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_rf.predict_proba(X_test)[:,1])

#svm_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, svm_model.predict_proba(X_test))
print("XGBoost_roc_auc", xgb_roc_auc)
print("LR_roc_auc", linear_roc_auc)
print("MLP_roc_auc", dnn_roc_auc)
print("RF_roc_auc", rf_roc_auc)
#print("svm_roc_auc", svm_roc_auc)
print()

xgb_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_rf.predict_proba(X_test)[:,1])
#svm_pr_auc = sklearn.metrics.average_precision_score(Y_test1, svm_model.predict_proba(X_test))
print("XGBoost_pr_auc", xgb_pr_auc)
print("LR_pr_auc", linear_pr_auc)
print("MLP_pr_auc", dnn_pr_auc)
print("RF_pr_auc", rf_pr_auc)
#print("svm_pr_auc", svm_pr_auc)
print()

xgb_fpr, xgb_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_fpr, linear_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_fpr, dnn_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_fpr, rf_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_rf.predict_proba(X_test)[:,1])
#svm_fpr, svm_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, svm_model.predict_proba(X_test))


#pl.plot(xgb_fpr, xgb_tpr, label="AUC XGBoost %.3f" % xgb_roc_auc)
pl.plot(rf_fpr, rf_tpr, label="AUC RF %.3f" % rf_roc_auc)
pl.plot(dnn_fpr, dnn_tpr, label="AUC MLP %.3f" % dnn_roc_auc)
pl.plot(linear_fpr, linear_tpr, label="AUC LR %.3f" % linear_roc_auc)
#pl.plot(svm_fpr, svm_tpr, label="SVM %f" % svm_roc_auc)
pl.legend()
pl.title("ROC curves on test data")
pl.show()

xgb_prec, xgb_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_xgb.predict_proba(X_test)[:,1])
dnn_prec, dnn_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_mlp.predict_proba(X_test)[:,1])
linear_prec, linear_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_lr.predict_proba(X_test)[:,1])
rf_prec, rf_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_rf.predict_proba(X_test)[:,1])
#svm_prec, svm_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, svm_model.predict_proba(X_test))

# pl.plot(xgb_recall, xgb_prec, label="XGBoost %.3f" % xgb_pr_auc)
pl.plot(dnn_recall, dnn_prec, label="MLP %.3f" % dnn_pr_auc)
pl.plot(linear_recall, linear_prec, label="LR %.3f" % linear_pr_auc)
pl.plot(rf_recall, rf_prec, label="RF %.3f" % rf_pr_auc)

#pl.plot(svm_recall, svm_prec, label="SVM %f" % svm_pr_auc)
pl.legend()
pl.title("Precision-Recall curves on test data")
pl.show()

In [ ]:
# 将测试集的预测结果提取出来
output_test_lr=pd.concat([test_data,pd.Series(model_lr.predict_proba(X_test)[:,1],name='probability_lr')], axis=1)
output_test_xgb=pd.concat([test_data,pd.Series(model_xgb.predict_proba(X_test)[:,1],name='probability_xgb')], axis=1)
output_test_mlp=pd.concat([test_data,pd.Series(model_mlp.predict_proba(X_test)[:,1],name='probability_mlp')], axis=1)
output_test_rf=pd.concat([test_data,pd.Series(model_rf.predict_proba(X_test)[:,1],name='probability_rf')], axis=1)



output_test_lr.to_csv('output_test_lr.csv', index=False) 
output_test_xgb.to_csv('output_test_xgb.csv', index=False) 
output_test_mlp.to_csv('output_test_mlp.csv', index=False) 
output_test_rf.to_csv('output_test_rf.csv', index=False) 



### LIME

In [ ]:
X_test = X_test.reset_index(drop=True)
X_test.iloc[0]

In [ ]:
# shap-rf

# 创建一个SHAP解释器
explainer = shap.TreeExplainer(model_rf_new)

# 计算测试集上的SHAP值
shap_values = explainer.shap_values(X_test)

# 将SHAP值转换为DataFrame
# 对于二分类问题，shap_values通常是一个列表，其中包含每个类别的SHAP值
# 我们通常关注正类的SHAP值，即shap_values[1]
feature_important_rf = pd.DataFrame(shap_values[1], columns=X_test.columns)
# 取绝对值
# 将结果保存到CSV文件
feature_important_rf.to_csv('feature_important_shap_rf_240328.csv', index=False)
# 保存DataFrame到CSV文件（如果需要）
feature_important_rf

In [ ]:
# 假设我们已经有了一个训练好的随机森林模型 `trained_rf_model`
# 和一个测试集 `X_test`。

# 创建LIME解释器
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),  # 使用训练数据的NumPy数组
    feature_names=X_train.columns.tolist(),  # 特征名称列表
    class_names=['Class 0', 'Class 1'],  # 类别名称
    mode='classification'  # 分类模式
)

# 定义需要提取的特征列表
str1 = ["Max_1HR", "Max_1HR_2", "Max_1HR_72", "Max_1TD", 'RH_Count', 'TD_Count', 'EV', 'TWI', 'DTW']

# 初始化一个空的DataFrame来存储特征贡献度
feature_importance_df = pd.DataFrame(columns=str1)

# 对测试集中的每个样本应用LIME并提取特征贡献度
for i in range(len(X_test)):
    # 使用LIME解释器解释每个实例
    exp = explainer.explain_instance(
        X_test.iloc[i].values, 
        model_rf_new.predict_proba, 
        num_features=len(str1)
    )
    
    # 提取特征贡献度并存入字典
    feature_importance_scores = {feature: 0 for feature in str1}  # 初始化所有特征的贡献度为0
    for feature, importance in exp.as_list():
        for feature_name in str1:
            if feature_name in feature:  # 检查特征名称是否存在于LIME返回的特征中
                feature_importance_scores[feature_name] += importance  # 累加贡献度

    # 将提取的特征贡献度添加到DataFrame中
    feature_importance_df.loc[len(feature_importance_df)] = feature_importance_scores

# 将结果保存到CSV文件
feature_importance_df.to_csv('feature_important_lime_rf_240328.csv', index=False)

# 打印DataFrame的前几行以检查结果
print(feature_importance_df.head())

In [ ]:
# lime-lr
X1_test=X_test
X1_test_np=X1_test.to_numpy()

# LIME——真

explainer = lime.lime_tabular.LimeTabularExplainer(X1_test_np ,class_names=['0', '1'], feature_names = X1_test.columns,
                                                    verbose=False)

# Create an empty DataFrame to store the feature importance scores
feature_names = X1_test.columns
feature_importance_df = pd.DataFrame(columns=feature_names)

# Loop over each test sample
for i in range(len(X1_test)):
    # Get the test sample
    test_sample = X1_test.iloc[i]
    # Explain the test sample using LIME
    exp = explainer.explain_instance(test_sample, model_lr_new.predict_proba, num_features=len(feature_names))
    str1 = ["Max_1HR", "Max_1HR_2", "Max_1HR_72", "Max_1TD", 'RH_Count','TD_Count','EV','TWI','DTW']
    dic = {}
    for s in range(exp.as_list().__len__()): # range(exp.as_list().__len__())=7 是因为有7个特征，所以range(exp.as_list().__len__())=7
        for i in str1:
            if i in exp.as_list()[s][0]: # exp.as_list()[s][0]是特征名
                dic[i] = exp.as_list()[s][1]
    # feature_importance_scores = dict(exp.as_list())
    feature_importance_scores = dic

    # Get the feature importance scores
    # feature_importance_scores = dict(exp.as_list())
    # Add the feature importance scores to the DataFrame
    feature_importance_df = pd.concat([feature_importance_df, pd.DataFrame([feature_importance_scores])], ignore_index=True)

feature_importance_df.to_csv('feature_importance_scores_LIME_lr_240315.csv')
print(feature_importance_df.head(10))




In [ ]:
# lime-xgb
X1_test=X_test
X1_test_np=X1_test.to_numpy()

# LIME——真

explainer = lime.lime_tabular.LimeTabularExplainer(X1_test_np ,class_names=['0', '1'], feature_names = X1_test.columns,
                                                    verbose=False)

# Create an empty DataFrame to store the feature importance scores
feature_names = X1_test.columns
feature_importance_df = pd.DataFrame(columns=feature_names)

# Loop over each test sample
for i in range(len(X1_test)):
    # Get the test sample
    test_sample = X1_test.iloc[i]
    # Explain the test sample using LIME
    exp = explainer.explain_instance(test_sample, model_xgb.predict_proba, num_features=len(feature_names))
    str1 = ["Max_1HR", "Max_1HR_2", "Max_1HR_72", "Max_1TD",'RH_Count','TD_Count','EV','TWI','DTW']
    dic = {}
    for s in range(exp.as_list().__len__()): # range(exp.as_list().__len__())=7 是因为有7个特征，所以range(exp.as_list().__len__())=7
        for i in str1:
            if i in exp.as_list()[s][0]: # exp.as_list()[s][0]是特征名
                dic[i] = exp.as_list()[s][1]
    # feature_importance_scores = dict(exp.as_list())
    feature_importance_scores = dic

    # Get the feature importance scores
    # feature_importance_scores = dict(exp.as_list())
    # Add the feature importance scores to the DataFrame
    feature_importance_df = pd.concat([feature_importance_df, pd.DataFrame([feature_importance_scores])], ignore_index=True)

feature_importance_df.to_csv(feature_importance_scores_LIME_xgb_240315.csv')
print(feature_importance_df.head(10))

In [ ]:
# lime-mlp
X1_test=X_test
X1_test_np=X1_test.to_numpy()

# LIME——真

explainer = lime.lime_tabular.LimeTabularExplainer(X1_test_np ,class_names=['0', '1'], feature_names = X1_test.columns,
                                                    verbose=False)

# Create an empty DataFrame to store the feature importance scores
feature_names = X1_test.columns
feature_importance_df = pd.DataFrame(columns=feature_names)

# Loop over each test sample
for i in range(len(X1_test)):
    # Get the test sample
    test_sample = X1_test.iloc[i]
    # Explain the test sample using LIME
    exp = explainer.explain_instance(test_sample, model_mlp.predict_proba, num_features=len(feature_names))
    str1 = ["Max_1HR",  "Max_1HR_2", "Max_1HR_72", "Max_1TD",'RH_Count','TD_Count','EV','TWI','DTW']
    dic = {}
    for s in range(exp.as_list().__len__()): # range(exp.as_list().__len__())=7 是因为有7个特征，所以range(exp.as_list().__len__())=7
        for i in str1:
            if i in exp.as_list()[s][0]: # exp.as_list()[s][0]是特征名
                dic[i] = exp.as_list()[s][1]
    # feature_importance_scores = dict(exp.as_list())
    feature_importance_scores = dic

    # Get the feature importance scores
    # feature_importance_scores = dict(exp.as_list())
    # Add the feature importance scores to the DataFrame
    feature_importance_df = pd.concat([feature_importance_df, pd.DataFrame([feature_importance_scores])], ignore_index=True)

feature_importance_df.to_csv('feature_importance_scores_LIME_mlp_240315.csv')
print(feature_importance_df.head(10))

### SHAP

In [ ]:
X_test

In [ ]:
# shap-lr

# Load your trained model here, assuming it's stored in a variable called 'model'
model = model_lr_new

# Compute SHAP values for all test samples
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test)

# Compute the sum of absolute SHAP values for each feature
feature_importance = np.abs(shap_values.values).mean(0)
feature_names = list(X_test.columns)

# Sort the features by the sum of absolute SHAP values
sorted_idx = np.argsort(feature_importance)[::-1]
sorted_features = [feature_names[i] for i in sorted_idx]

# Create a DataFrame to store the feature rankings
df = pd.DataFrame(columns=sorted_features)

# Add each sample's feature rankings as a row in the DataFrame
for i in range(X_test.shape[0]):
    row = shap_values[i, sorted_idx].values.reshape(1, -1)
    # 原来的代码
    # df = df.append(pd.DataFrame(row, columns=sorted_features), ignore_index=True)

    # 使用 pd.concat 替代
    new_row = pd.DataFrame(row, columns=sorted_features)
    df = pd.concat([df, new_row]).reset_index(drop=True)

df_test_shap_lr = df
# Print the DataFrame
print(df)
df_test_shap_lr.to_csv('feature_importance_scores_SHAP_lr_240315.csv', index=False)

In [ ]:
# shap-xgboost
# 计算的是特征贡献值的绝对值

# Compute SHAP values for all test samples.
explainer = shap.Explainer(model_xgb_new)
shap_values_11 = explainer(X_test)

# Compute the sum of absolute SHAP values for each feature.
shap_values_sum = np.abs(shap_values_11.values).sum(axis=0)

feature_names = X_test.columns
# Sort the features by the sum of absolute SHAP values.
sorted_indices = np.argsort(shap_values_sum)[::-1]
sorted_features = feature_names[sorted_indices]

# Create a DataFrame to store the feature rankings.
df_test_shap_xgb = pd.DataFrame(columns=sorted_features)

# Add each sample's feature rankings as a row in the DataFrame.
for i in range(len(X_test)):
    row = np.abs(shap_values_11.values[i, sorted_indices]).tolist()
    df_test_shap_xgb.loc[i] = row

# Print the DataFrame.
print(df_test_shap_xgb)
# 将DF保存到CSV
df_test_shap_xgb.to_csv('feature_importance_scores_SHAP_xgb_240315.csv', index=False)

In [ ]:
# Load the MLP model and test data
mlp_model = model_mlp_new # load the MLP model


# Calculate SHAP values for the test set using the MLP model.
explainer = shap.Explainer(mlp_model.predict, X_train)
shap_values = explainer(X_test)

# Compute the sum of absolute SHAP values for each feature.
feature_importances = np.abs(shap_values.values).sum(axis=0)

# Sort the feature importances for each sample.
sorted_indices = np.argsort(-np.abs(shap_values.values), axis=1)

# Create a DataFrame to store the feature importances.
df_feature_importances = pd.DataFrame(columns=X_test.columns)

# Add each sample's feature rankings as a row in the DataFrame.
for i in range(len(X_test)):
    row = np.abs(shap_values.values[i, sorted_indices[i]]).tolist()
    df_feature_importances.loc[i] = row

# Print the DataFrame.
print(df_feature_importances)
df_test_shap_mlp = df_feature_importances
# 将DF保存到CSV
df_test_shap_mlp.to_csv('feature_importance_scores_SHAP_mlp_240315.csv', index=False)